# python-mlir-graphblas

Notebook for trying out the functionality as it is built

In [ ]:
import mlir_graphblas as mlgb
from mlir_graphblas import types, tensor, operators, operations
from mlir_graphblas import descriptor as desc

In [ ]:
v = tensor.Vector.new(types.FP32, 8)
v.build([0, 2, 3, 5], [1, 2, 3, 4])
w = tensor.Vector.new(types.FP64, 8)
w.build([0, 2, 4, 5, 6], [2, 3, 4, .5, 6])

y = tensor.Vector.new(types.FP32, 8)
operations.ewise_add(y, operators.BinaryOp.first, v, w)
y.extract_tuples()

In [ ]:
z = tensor.Vector.new(types.FP32, 8)
operations.ewise_mult(z, operators.BinaryOp.oneb, v, w)
z.extract_tuples()

In [ ]:
b = tensor.Vector.new(types.BOOL, 8)
operations.ewise_mult(b, operators.BinaryOp.gt, v, w)
b.extract_tuples()

In [ ]:
x = tensor.Matrix.new(types.FP32, 3, 3)
x.build([0, 0, 1, 1, 2], [1, 2, 0, 2, 1], [1, 2, 3, 4, 5], sparsity=["dense", "compressed"])
y = tensor.Matrix.new(types.FP32, 3, 3)
y.build([0, 0, 1, 1, 2], [1, 2, 0, 2, 1], [1, 2, 3, 4, 5], sparsity=["compressed", "compressed"])
z = tensor.Matrix.new(types.FP32, 3, 3)

In [ ]:
operations.ewise_add(z, operators.BinaryOp.plus, y, x)

In [ ]:
z

In [ ]:
z.extract_tuples()

#### apply mask

In [ ]:
from mlir_graphblas import implementations as impl

mask = tensor.Vector.new(types.FP64, 8)
mask.build([0, 3, 6], [-1., 2.2, 6.28318])

bb = impl.select_by_mask(v, mask, desc.SC)
bb.extract_tuples()

#### eWiseAdd Matrix variants

In [ ]:
import numpy as np
T = types.INT64

def view_sizes(x):
    print(mlgb.utils.c_lib.sparseDimSize(x._obj[0], 0), mlgb.utils.c_lib.sparseDimSize(x._obj[0], 1))
    print(mlgb.utils.c_lib.sparseLvlSize(x._obj[0], 0), mlgb.utils.c_lib.sparseLvlSize(x._obj[0], 1))

m1 = tensor.Matrix.new(T, 7, 5)
m1.build(np.array([0, 0, 0, 0, 0]),
         np.array([0, 1, 2, 3, 4]),
         np.array([1, 2, 3, 4, 5], dtype=T.np_type), colwise=False)
view_sizes(m1)
m1

In [ ]:
m2 = tensor.Matrix.new(T, 7, 5)
m2.build([0, 1, 2, 3, 4, 5, 6],
         [3, 3, 3, 3, 3, 3, 3],
         [10, 20, 30, 40, 50, 60, 70], colwise=False)
view_sizes(m2)
m2

In [ ]:
m_out = tensor.Matrix.new(T, 7, 5)
operations.ewise_add(m_out, operators.BinaryOp.plus, m1, m2)
view_sizes(m_out)
print(m_out)
m_out.extract_tuples()

In [ ]:
m_out = tensor.Matrix.new(T, 5, 7)
operations.ewise_add(m_out, operators.BinaryOp.plus, m1, m2, desc=desc.T0T1)
view_sizes(m_out)
print(m_out)
m_out.extract_tuples()

## Things I learned

1. indexing_map and dimOrdering are very different
2. indexing_map is used to transpose an object (doesn't affect iteration order)
3. dimOrdering affect iteration order
4. Secondary matrix can have any iteration order
5. Iteration order of first matrix must match output matrix
6. That matching can be transposed and inverted dims together (i.e. transposing a rowwise input works for a non-transposed colwise output); we need to fix the ordering declared (see above)

#### Mini Mystery

Crash happens if sizes get too large. Not sure why.

#### mxm

In [ ]:
import mlir_graphblas as mlgb
from mlir_graphblas import types, tensor, operators, operations
from mlir_graphblas import descriptor as desc

In [ ]:
import numpy as np

r1 = np.array([[11., 10., 0., -4.],
               [0., 0., 0., 0.],
               [0., 0., 0., 0.]])
r2 = np.array([[4.1, 0., 2.3],
               [10., 0., 0.],
               [0., 0., 0.],
               [0., -100., 0.]])
r1 @ r2

In [ ]:
m1 = tensor.Matrix.new(types.FP64, 3, 4)
m2 = tensor.Matrix.new(types.FP64, 4, 3)

m1.build([0, 0, 0], [0, 1, 3], [11., 10., -4.], colwise=False, sparsity=["compressed", "compressed"])
m2.build([0, 0, 1, 3], [0, 2, 0, 1], [4.1, 2.3, 10.0, -100.0], colwise=False, sparsity=["dense", "compressed"])

m_out = tensor.Matrix.new(types.FP64, 3, 3)

operations.mxm(m_out, operators.Semiring.plus_times, m1, m2)
print(m_out)
m_out.extract_tuples()

In [ ]:
r2 @ r1

In [ ]:
m2_out = tensor.Matrix.new(types.FP64, 4, 4)
operations.mxm(m2_out, operators.Semiring.plus_times, m1, m2, desc=desc.T0T1)
print(m2_out)
m2_out.extract_tuples()

#### mxv

In [ ]:
m = tensor.Matrix.new(types.FP64, 3, 4)
v = tensor.Vector.new(types.FP64, 4)

m.build([0, 0, 0], [0, 1, 3], [11., 10., -4.], colwise=False)
v.build([0, 3], [4.1, 2.3])

m_out = tensor.Vector.new(types.FP64, 3)

operations.mxv(m_out, operators.Semiring.plus_times, m, v)
print(m_out)
m_out.extract_tuples()

In [ ]:
import numpy as np
v = np.array([[4.1, 0., 0., 2.3]])
m = np.array([[11., 10., 0., -4], [0, 0, 0, 0], [0, 0, 0, 0]])
m @ v.T

#### vxm

In [ ]:
m = tensor.Matrix.new(types.FP64, 3, 4)
v = tensor.Vector.new(types.FP64, 3)

m.build([0, 0, 0], [0, 1, 3], [11., 10., -4.], colwise=False)
v.build([0, 2], [4.1, 2.3])

m_out = tensor.Vector.new(types.FP64, 4)

operations.vxm(m_out, operators.Semiring.plus_times, v, m)
print(m_out.size())
m_out.extract_tuples()

In [ ]:
import numpy as np
v = np.array([[4.1, 0., 2.3]])
m = np.array([[11., 10., 0., -4], [0, 0, 0, 0], [0, 0, 0, 0]])
v @ m

#### select

In [ ]:
import mlir_graphblas as mlgb
from mlir_graphblas import types, tensor, operators, operations
from mlir_graphblas import descriptor as desc

In [ ]:
m3 = tensor.Matrix.new(types.FP64, 4, 3)
m3.build([0, 0, 1, 3], [0, 2, 0, 1], [4.1, 2.3, 10.0, -100.0])

m_out = tensor.Matrix.new(types.FP64, 4, 3)

In [ ]:
operations.select(m_out, operators.SelectOp.tril, m3, 0)
m_out.extract_tuples()

In [ ]:
operations.select(m_out, operators.SelectOp.valuegt, m3, 3.3)
m_out.extract_tuples()

In [ ]:
m4 = tensor.Matrix.new(types.FP32, 16, 3)
m4.build([2], [2], [0.1])
m_out = tensor.Matrix.new(types.FP32, 16, 3)

operations.select(m_out, operators.SelectOp.valueeq, m4, 0.0)
m_out.extract_tuples()

#### apply

In [ ]:
# In-place
m3 = tensor.Matrix.new(types.FP64, 4, 3)
m3.build([0, 0, 1, 3], [0, 2, 0, 1], [4.1, 2.3, 10.0, -100.0])
operations.apply(m3, operators.BinaryOp.div, m3, right=2)
m3.extract_tuples()

In [ ]:
# Apply Binary second (val / 2)
m3 = tensor.Matrix.new(types.FP64, 4, 3)
m3.build([0, 0, 1, 3], [0, 2, 0, 1], [4.1, 2.3, 10.0, -100.0])
m_out = tensor.Matrix.new(types.FP64, 4, 3)
operations.apply(m_out, operators.BinaryOp.div, m3, right=2)
m_out.extract_tuples()

In [ ]:
# Apply Binary first (2 / val)
m3 = tensor.Matrix.new(types.FP64, 4, 3)
m3.build([0, 0, 1, 3], [0, 2, 0, 1], [4.1, 2.3, 10.0, -100.0])
m_out = tensor.Matrix.new(types.FP64, 4, 3)
operations.apply(m_out, operators.BinaryOp.div, m3, left=2)
m_out.extract_tuples()

In [ ]:
# Apply Unary
m3 = tensor.Matrix.new(types.FP64, 4, 3)
m3.build([0, 0, 1, 3], [0, 2, 0, 1], [4.1, 2.3, 10.0, -100.0])
m_out = tensor.Matrix.new(types.FP64, 4, 3)
operations.apply(m_out, operators.UnaryOp.ainv, m3)
m_out.extract_tuples()

In [ ]:
# Apply IndexUnaryOp (col_index + 10)
m3 = tensor.Matrix.new(types.FP64, 4, 3)
m3.build([0, 0, 1, 3], [0, 2, 0, 1], [4.1, 2.3, 10.0, -100.0])
m_out = tensor.Matrix.new(types.INT64, 4, 3)
operations.apply(m_out, operators.IndexUnaryOp.colindex, m3, thunk=10)
m_out.extract_tuples()

#### reduce to vector

In [ ]:
import mlir_graphblas as mlgb
from mlir_graphblas import types, tensor, operators, operations
from mlir_graphblas import descriptor as desc

In [ ]:
# Rowwise
m = tensor.Matrix.new(types.FP64, 4, 3)
m.build([0, 0, 1, 3], [0, 2, 0, 2], [4.1, 2.3, 10.0, -100.0])
v_out = tensor.Vector.new(types.FP64, 4)
operations.reduce_to_vector(v_out, operators.Monoid.plus, m)
v_out.extract_tuples()

In [ ]:
# Colwise
v_out = tensor.Vector.new(types.FP64, 3)
operations.reduce_to_vector(v_out, operators.Monoid.times, m, desc=desc.T0)
v_out.extract_tuples()

In [ ]:
# Rowwise with mask
mask = tensor.Vector.new(types.BOOL, 4)
mask.build([0, 1], [1, 1])
v_out = tensor.Vector.new(types.FP64, 4)
operations.reduce_to_vector(v_out, operators.Monoid.plus, m, mask=mask, desc=desc.S)
v_out.extract_tuples()

In [ ]:
# Colwise with complemented mask
mask = tensor.Vector.new(types.BOOL, 3)
mask.build([0, 1], [1, 1])
v_out = tensor.Vector.new(types.FP64, 3)
operations.reduce_to_vector(v_out, operators.Monoid.times, m, mask=mask, desc=desc.SCT0)
v_out.extract_tuples()

#### reduce to scalar

In [ ]:
# Matrix
m = tensor.Matrix.new(types.FP64, 4, 3)
m.build([0, 0, 1, 3], [0, 2, 0, 2], [4.1, 2.3, 10.0, -100.0])
s_out = tensor.Scalar.new(types.FP64)
operations.reduce_to_scalar(s_out, operators.Monoid.plus, m)
s_out

In [ ]:
# Vector
v = tensor.Vector.new(types.INT16, 120)
v.build([13, 54, 97, 101], [17, 13, 14, 15])
s_out = tensor.Scalar.new(types.INT16)
operations.reduce_to_scalar(s_out, operators.Monoid.min, v)
s_out

#### extract

In [ ]:
# Vector
v = tensor.Vector.new(types.INT16, 10)
v.build([1, 3, 4, 7], [17, 13, 14, 15])
z = tensor.Vector.new(types.INT16, 4)
operations.extract(z, v, [1, 2, 4, 5])
z.extract_tuples()

In [ ]:
# Vector extract GrB_ALL
z = tensor.Vector.new(types.INT16, 10)
operations.extract(z, v)
z.extract_tuples()

In [ ]:
# Matrix
m = tensor.Matrix.new(types.FP32, 3, 4)
m.build([0, 0, 0, 1, 1, 1, 2, 2, 2], [0, 1, 3, 0, 2, 3, 0, 1, 2], [1, 2, 3, 4, 5, 6, 7, 8, 9])
z = tensor.Matrix.new(types.FP32, 2, 2)
operations.extract(z, m, [1, 2], [0, 3])
z.extract_tuples()

In [ ]:
# Matrix extract column
z = tensor.Vector.new(types.FP32, 2)
operations.extract(z, m, [1, 2], 3)
z.extract_tuples()

In [ ]:
# Matrix extract row
z = tensor.Vector.new(types.FP32, 2)
operations.extract(z, m, 1, [0, 3])
z.extract_tuples()

In [ ]:
# Matrix extract GrB_ALL
z = tensor.Matrix.new(types.FP32, 3, 4)
operations.extract(z, m)
z.extract_tuples()